### GPU

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### IMPORTS

In [2]:
import torch
import torchvision
import PIL
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from autopilot_dataset import AutopilotDataset
from autopilot_model import AutopilotModel
from autopilot_utils import preprocess_image

<br>

### HYPERPARAMETERS

In [3]:
BATCH_SIZE = 128
MAX_EPOCHS = 50
EARLY_STOPPING_PATIENCE = 10

INITIAL_LR = 0.0005
LR_REDUCER_PATIENCE = 2
LR_REDUCER_FACTOR = 0.9

ACCEPTABLE_TESTING_LOSS = 0.1

FRAME_SIZE = 224

MODELS_DIR = "/home/greg/models/jetson/"
DATASETS_DIR = "/home/greg/datasets/jetson/"

VERSION = "2_16"

MODEL_PATH = MODELS_DIR + VERSION + "_resnet18" + ".pth"
TRAINING_DATASET = DATASETS_DIR + "training/"
VALIDATION_DATASET = DATASETS_DIR + "validation/"
TESTING_DATASET = DATASETS_DIR + "testing/"

<br>

### DATA

In [ ]:
training_dataset = AutopilotDataset(TRAINING_DATASET,
                                    FRAME_SIZE,
                                    random_horizontal_flip=True,
                                    random_noise=True,
                                    random_blur=True,
                                    random_color_jitter=True,
                                    keep_images_in_ram=True)
training_loader = torch.utils.data.DataLoader(training_dataset,
                                              batch_size=BATCH_SIZE,
                                              shuffle=True)

validation_dataset = AutopilotDataset(VALIDATION_DATASET,
                                      FRAME_SIZE,
                                      random_horizontal_flip=False,
                                      random_noise=False,
                                      random_blur=False,
                                      random_color_jitter=False,
                                      keep_images_in_ram=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True)

testing_dataset = AutopilotDataset(TESTING_DATASET,
                                   FRAME_SIZE,
                                   random_horizontal_flip=False,
                                   random_noise=False,
                                   random_blur=False,
                                   random_color_jitter=False,
                                   keep_images_in_ram=True)
testing_loader = torch.utils.data.DataLoader(testing_dataset,
                                                batch_size=1,
                                                shuffle=False)

<br>

### MODEL

In [5]:
model = AutopilotModel(pretrained=True)
optimizer = torch.optim.Adam(model.parameters(), lr=INITIAL_LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       'min',
                                                       patience=LR_REDUCER_PATIENCE,
                                                       factor=LR_REDUCER_FACTOR,
                                                       verbose=True)  
loss_function = torch.nn.MSELoss()

<br>

### TRAINING

In [ ]:
training_losses = []
validation_losses = []
epochs_without_improvement = 0

def plot_losses():
    fig, ax = plt.subplots()
    ax.plot([x for x in range(len(training_losses))], training_losses, label='training_loss')
    ax.plot([x for x in range(len(validation_losses))], validation_losses, label='validation_loss')
    ax.set(xlabel='epochs', ylabel='loss', title='Training Progress')
    ax.grid()
    plt.legend()
    plt.show()
    
def run_epoch(tepoch, name, training):
    epoch_loss = 0.0
    iterations = 0
    
    for _, images, annotations in tepoch:
        tepoch.set_description(f"{name} Epoch {epoch}")

        images = images.cuda()
        annotations = annotations.cuda()
        
        if training:
            optimizer.zero_grad()
            model.train()
            outputs = model(images)
        else:
            with torch.no_grad():
                model.eval()
                outputs = model(images)
        
        loss = loss_function(outputs, annotations)
        
        if training:
            loss.backward()
            optimizer.step()

        epoch_loss += loss.item()
        iterations += 1
        
    return float(epoch_loss/iterations)

for epoch in range(MAX_EPOCHS):
    with tqdm(training_loader, unit="batch") as training_epoch:
        avg_training_loss = run_epoch(training_epoch, "Training", training=True)
        training_losses.append(avg_training_loss)
    
    with tqdm(validation_loader, unit="batch") as validation_epoch:
        avg_validation_loss = run_epoch(validation_epoch, "Validation", training=False)
        scheduler.step(avg_validation_loss)
        validation_losses.append(avg_validation_loss)
        
        if avg_validation_loss <= np.min(validation_losses):
            epochs_without_improvement = 0
            print("validation loss decreased to " + str(avg_validation_loss) + ", saving model")
            model.save_to_path(MODEL_PATH)
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= EARLY_STOPPING_PATIENCE:
                print("validation loss of " + str(np.min(validation_losses)) + " hasn't improved in last " + str(EARLY_STOPPING_PATIENCE) + " epochs, stopping training")
                break
        
    plot_losses()

<br>

### TESTING

In [ ]:
model.load_from_path(MODEL_PATH)

results = []
losses = []
with torch.no_grad():
    model.eval()
    
    for name, image, annotation in testing_loader:
        prediction = model(image.cuda()).clamp(min=-1, max=1)
        loss = round(float(loss_function(prediction, annotation.cuda()).cpu()), 4)
        passed = loss < ACCEPTABLE_TESTING_LOSS
        losses.append(loss)
        results.append(passed)
               
        composed_transforms = torchvision.transforms.Compose([
            torchvision.transforms.Normalize([-0.485/0.229, -0.456/0.224, -0.406/0.225], [1/0.229, 1/0.224, 1/0.225]),
            torchvision.transforms.ToPILImage()
        ])
        image = composed_transforms(image[0])
        image = image.convert("RGB")
        display(image)
        
        print(name[0])
        print("expected: "+str(annotation.float()[0]))
        print("predicted: "+str(prediction.cpu().float()[0]))
        print("loss: "+str(loss))
        print("passed: "+str(passed))
        print("")

print("SCORE: "+str(len([x for x in results if x]))+"/"+str(len(results))+", AVG LOSS: "+str(round(np.mean(losses), 4)))    